In [5]:
import pandas as pd
import numpy as np
import requests
import os
from pathlib import Path

notebook_path = os.path.join(os.getcwd())
data_path = Path(notebook_path).parent.joinpath('data')

# Read preprocessed data
df = pd.read_csv(data_path / 'wildfires_preprocessed.csv', parse_dates=True)

# We'll use units IDs for forward geofencing
units = df.UNIT_ID.unique()


'''
We'll use Radar API for forward geofencing. According to its documentation:

curl "https://api.radar.io/v1/search/autocomplete?query=841+Broadway&near=40.70390%2C-73.98670" -H 
"Authorization: prj_live_pk_4f017205dfaad39af3182404866753da475d1e04"
Response:

{'latitude': 34.05513,
 'longitude': -118.25703,
 'geometry': {'type': 'Point', 'coordinates': [-118.25703, 34.05513]},
 'country': 'United States',
 'countryCode': 'US',
 'countryFlag': '🇺🇸',
 'county': 'Los Angeles County',
 'distance': 2809386,
 'city': 'Los Angeles',
 'stateCode': 'CA',
 'state': 'California',
 'layer': 'locality',
 'formattedAddress': 'Los Angeles, CA USA',
 'addressLabel': 'Los Angeles'}
 '''

# Go through each Unit ID and request forward geofencing via API
headers = {'Authorization': 'prj_live_pk_4f017205dfaad39af3182404866753da475d1e04'}
coords_lat = []
coords_lon = []
for i, unit in enumerate(units):
    try:  # try forward geofencing
        query = unit.replace('CAL FIRE', 'calfire').replace('NP', 'National Park').replace('BLM', 'Bureau of Land Management').replace(' - ', '-').replace(' FD', ' Fire Department').replace(' - ', '-')
        query += ' California, USA'
        url = f"https://api.radar.io/v1/search/autocomplete?query={query}"
        print(f'{i}.Requesting: {query}')
        response = requests.get(url, headers=headers)
        assert response.status_code == 200, f'error: {response.status_code}'
        assert len(response.json()['addresses'])
    except:  # try more versions of search query, sometimes it helps to remove 'calfire'
        try:
            query = unit.replace('CAL FIRE', 'calfire').replace('NP', 'National Park').replace('BLM', 'Bureau of Land Management').replace(' - ', '-').replace('calfire','')
            query += ' California, USA'
            url = f"https://api.radar.io/v1/search/autocomplete?query={query}"
            print(f'{i}.Requesting: {query}')
            response = requests.get(url, headers=headers)
            assert response.status_code == 200, f'error: {response.status_code}'
            assert len(response.json()['addresses'])
        except:  # if nothing found, save as 0, 0
            print('got NaN')
            coords_lat.append(0)
            coords_lon.append(0)
            continue
    lat = response.json()['addresses'][0]['latitude']
    lon = response.json()['addresses'][0]['longitude']
    coords_lat.append(lat)
    coords_lon.append(lon)
    print(f'Got: \t{unit}: {lat}, {lon}')
    
# Construct DataFrame with units and corresponding coordinates
units_df = pd.DataFrame(data={'unit':units,'lat':coords_lat,'lon':coords_lon})

# Save
units_df.to_csv(data_path / 'units_geocoordinates2.csv', index=False)



'''
Notes:
The following fires don't have location:
CRA unit - unknown  
BARRY POINT - oregon  
OREGON GULCH - oregon  
LOCUST - North Sacramento County

The following units don't have correct coordinates:
Carson City District or - BLM
Humbolt-Toiyabe National Forest
Mojave - NPS
CRA
Rouge River- Siskiyou
NaN
'''

# Merge main preprocessed DataFrame and GeoCoordinates Dataframe
df = df.merge(units_df, left_on='UNIT_ID',right_on='unit')

# Drop missing data rows and rows with wrong coordinates
# ToDo: try to find manually
df = df.dropna(subset='UNIT_ID')
df = df[~df.UNIT_ID.isin(['CRA', 'Carson City District or - BLM', 'Humbolt-Toiyabe National Forest', 'Mojave - NPS', 'Rouge River- Siskiyou'])]
df = df[df.lat!=0]

# Rename columns
df.columns = [x.lower() for x in df.columns]

# Save
df.to_csv(data_path / 'wildfires_with_geocoord.csv', index=False)

0.Requesting: Los Angeles County California, USA
Got: 	Los Angeles County: 34.05513, -118.25703
1.Requesting: Ventura County California, USA
Got: 	Ventura County: 34.27834, -119.29317
2.Requesting: Riverside calfire California, USA
Got: 	Riverside CAL FIRE: 33.496892, -117.1504
3.Requesting: Shasta-Trinity calfire California, USA
Got: 	Shasta - Trinity CAL FIRE: 40.546793, -122.3855
4.Requesting: San Bernardino National Forest California, USA
Got: 	San Bernardino National Forest: 34.090473, -117.260245
5.Requesting: Plumas National Forest California, USA
Got: 	Plumas National Forest: 39.937816, -120.94114
6.Requesting: Fresno-Kings calfire California, USA
Got: 	Fresno-Kings CAL FIRE: 36.75156, -119.800612
7.Requesting: Humboldt-Del Norte calfire California, USA
Got: 	Humboldt - Del Norte CAL FIRE: 40.641635, -123.926104
8.Requesting: Tahoe National Forest California, USA
Got: 	Tahoe National Forest: 39.301182, -120.678146
9.Requesting: Stanislaus National Forest California, USA
Got: 	S